In [12]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import openai
import pinecone
from pinecone import Pinecone, ServerlessSpec


In [15]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index( 
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws",region="us-east-1")
)

In [18]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Alice Johnson',
  'subject': 'Introduction to Psychology',
  'stars': 5,
  'review': 'Dr. Johnson is an excellent professor who makes complex concepts easy to understand. Her lectures are engaging and she is very approachable for help.'},
 {'professor': 'Professor Bob Smith',
  'subject': 'Calculus I',
  'stars': 4,
  'review': 'Professor Smith is knowledgeable and provides clear explanations. However, his assignments can be quite challenging and there is little feedback on grading.'},
 {'professor': 'Dr. Clara Nguyen',
  'subject': 'Organic Chemistry',
  'stars': 3,
  'review': "Dr. Nguyen's class is informative but can be a bit disorganized. She tends to rush through material and the exams are very difficult."},
 {'professor': 'Dr. David Lee',
  'subject': 'Modern Literature',
  'stars': 4,
  'review': 'Dr. Lee is passionate about literature and his discussions are always stimulating. The coursework is intensive but rewarding.'},
 {'professor': 'Professor Emily Cl

In [24]:
from openai import OpenAI


processd_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model = "text-embedding-3-nall",
    )
    
    embedding = response.data[0].embedding
    processd_data.append({
    "values": embedding,
    "id": review["professor"],
    "metadata": {
        "review": review['review']
    }
})

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-or-v1*************************************************************c6e8. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [25]:

index = pc.Index('rag')
index.upsert(
    vectors=processd_data,
    namespace="ns1"
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 24 Aug 2024 21:51:00 GMT', 'Content-Type': 'application/json', 'Content-Length': '74', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '45', 'x-pinecone-request-id': '2313991395812133150', 'x-envoy-upstream-service-time': '46', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"No vectors provided for upsert request","details":[]}


In [ ]:
index.describe_index_stats()